<h1 align=center><font size = 5>Machine Learning models for aiding the decision-making process in emergency departments</font></h1>

<h1>Tabla comparativa de algoritmos<h1>
<h3>Descripción</h3>
<p>
En este script se desarrollará una serie de predicciones basadas en datos del hospital San Juan de Dios Curicó, correspondientes al año 2018 representados por registros de urgencias. El objetivo es predecir, mediante un conjunto de algoritmos, la necesidad de hospitalización de un paciente de urgencias,tomando como input, datos proporcionados por el paciente en la etapa de registro, sus signos vitales registrados en la etapa de triage y el diagnóstico ofrecido por el médico tratante.
Se correrán algoritmos de prodicción tales como árboles y bosques de desición, regresión logística, support vector machines y redes neuronales. Para finalmente evaluar el rendimiento de cada algoritmo en términos de la predicción, mediante indicadores tales como Acuraccy, F1-Score, Curva ROC, Índice de Jaccard y logloss
</p>

<h1 id="Descripción de datos">Descripción de datos</h1>
<p>
Los datos utilizados fueros proporcionados por el Hospital San Juan de Dios, Curicó, Chile y corresponden a 4.971 registros de pacientes que asistieron a urgencias durante el periodo comprendido entre el 1 de enero de 2018 y agosto de 2019, los datos fueron limpiados y transformados en un script desarrollado previamente
<ul>
    <li>Datos: <a href="https://drive.google.com/open?id=1Bp7_MnK6cGwgBuwIq1a8S4DS_0wVDiAD" target="_blank">https://drive.google.com/open?id=1Bp7_MnK6cGwgBuwIq1a8S4DS_0wVDiAD</a></li>
    <li>Tipo de datos: csv</li>
   </ul>
<p>
Las variables presentes en los datos se describen a continuación:
<ul>    
   
   <li><b>PAC_EDAD</b>: corresponde a la edad del paciente en enteros</li>
   <li><b>MOTIVO_CONSULTA</b>: corresponde a la razón por la que el paciente acude al servicio de urgencias string</li>
   <li><b>MEDIO</b>: corresponde al medio de llegada, mediante el que el paciente acude al servicio de urgencias</li>
   <li><b>SEXO</b>: corresponde al sexo del paciente</li>
   <li><b>CAT</b>: corresponde a la categoría de gravedad asignada al paciente en el proceso de Triage</li>
   <li><b>PRESION_SIST</b>: corresponde la presión sistólica del paciente </li>
   <li><b>PRESION_DIAST</b>: corresponde la presióndiastólica del paciente</li>
   <li><b>SATO2</b>: Dato numérico que representa la saturometria del paciente (Nivel de oxigeno en la sangre)</li>
   <li><b>TEMPERATURA</b>: corresponde a la temperatura corporal del paciente en el momento de la categorización</li>
   <li><b>GLASGOW</b>: corresponde a al nivel registrado por el paciente en la escala Glasgow</li>
   <li><b>DM</b>: corresponde si el paciente presenta o no Diabetes Mellitus</li>
   <li><b>EVA</b>: corresponde si se aplica al paciente una evaluación de vias aéreas</li>
   <li><b>HGT</b>: corresponde a la medida de azucar en la sangre del paciente</li>
   <li><b>LCFA</b>: corresponde a si el paciente presenta obstrucción crónica de vías aéreas</li>
   <li><b>FR</b>: corresponde a la frecuencia respiratoria del paciente</li>
   <li><b>HTA</b>: corresponde a si el paciente posee Hipertención Arterial</li>
   <li><b>HORA_INSC</b>: corresponde a la hora en la que el paciente fue categorizado</li>
   <li><b>MIN_INSC</b>: corresponde al minuto en que el paciente fue categorizado</li>
   <li><b>TIEMPO_ESPERA_CAT</b>: corresponde al tiempo que espera el paciente luego de ser registrado, para ser categorizado</li>
      
</ul>
</p>

Cargar paquetes necesarios

In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
import sklearn as sk  
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
#Métodos de tuneo de parámetros
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Lectura de datos

In [2]:
archivo= 'df_limpia_hosp.csv'
df_urg= pd.read_csv(archivo,encoding='latin-1',sep=",")

In [3]:
df_urg.columns

Index(['Unnamed: 0', 'PAC_EDAD', 'COMUNA', 'PREVISION', 'MOTIVO_CONSULTA',
       'MEDIO', 'DESC_EVENTO', 'SEXO', 'SATO2', 'TEMPERATURA', 'GLASGOW', 'DM',
       'EVA', 'HGT', 'LCFA', 'FR', 'HTA', 'CAT', 'DESTINO', 'VIOLENCIA',
       'TIEMPO_ESPERA_CAT', 'TIEMPO_ESPERA_ATENCION', 'DURACION_ATENCION',
       'MES_INSC', 'SEMANA_INSC', 'DIA_MES_INSC', 'DIA_SEMANA_INSC',
       'HORA_INSC', 'MIN_INSC', 'DIA_MES_FIN_ATENCION',
       'DIA_SEMANA_FIN_ATENCION', 'HORA_INSC_FIN_ATENCION',
       'MIN_INSC_FIN_ATENCION', 'PRESION_SIST', 'PRESION_DIAST'],
      dtype='object')

Seleccionar las variables relevantes para la predicción y asignarselas a la matriz X, de variables independientes

In [4]:
df_hosp = df_urg[["PAC_EDAD",'MOTIVO_CONSULTA','MEDIO', 'SEXO','DESC_EVENTO','GLASGOW', 'DM', 'EVA','HGT', 'LCFA', 'FR', 
                 'HTA','CAT','TIEMPO_ESPERA_CAT','MES_INSC','DIA_MES_INSC','SEMANA_INSC','DIA_SEMANA_INSC','HORA_INSC',
                 'MIN_INSC','PRESION_SIST','PRESION_DIAST','TIEMPO_ESPERA_CAT','TIEMPO_ESPERA_ATENCION','DURACION_ATENCION','DESTINO']]

In [5]:
X = df_hosp[["PAC_EDAD",'MOTIVO_CONSULTA','MEDIO', 'SEXO','GLASGOW', 'DM', 'EVA','HGT', 'LCFA', 'FR', 
                 'HTA','CAT','TIEMPO_ESPERA_CAT','MES_INSC','DIA_MES_INSC','SEMANA_INSC','DIA_SEMANA_INSC','HORA_INSC',
                 'MIN_INSC','PRESION_SIST','PRESION_DIAST','TIEMPO_ESPERA_CAT','TIEMPO_ESPERA_ATENCION','DURACION_ATENCION']].values

In [6]:
X[0:2]

array([[79, 'OTROS, ESPECIFICAR', 'POR SUS PROPIOS MEDIOS', 'FEMENINO',
        15.0, 'S', 3.0, 121.0, 'S', 14.0, 'S', 'C3', 11, 11, 5, 23, 21,
        2, 17, 1, 159.0, 64.0, 11, 11, 93, 268],
       [36, 'OTROS, ESPECIFICAR', 'SAMU MOVIL AVANZADO', 'MASCULINO',
        15.0, 'N', 5.0, 114.0, 'N', 17.0, 'N', 'C3', 3, 3, 5, 17, 20, 3,
        22, 18, 131.0, 84.0, 3, 3, 14, 209]], dtype=object)

 <h1 id="Preprocesamiento de datos">Preprocesamiento de datos</h1>
<p>
Para aplicar el paquete de arboles de desición, los datos deben ser numéricos, en este caso siguen siendo en su mayoría categoricos pero serán transformados a nominales.
</p>

In [7]:
from sklearn import preprocessing

Preprocesando motivo de consulta

In [8]:
le_motivo_consulta = preprocessing.LabelEncoder()
le_motivo_consulta.fit(['OTROS, ESPECIFICAR','DOLOR Y/O MOLESTIAS ABDOMEN'
,'PROBLEMAS RESPIRATORIOS','DOLOR Y/O MOLESTIAS TORAX','DOLOR Y/O MOLESTIAS EXTREMIDADES',
'ACCIDENTE/TRAUMATISMO OTRO TIPO','ACCIDENTE/TRAUMATISMO EN EL HOGAR', 
'DIABETES MELLITUS DESCOMPENSADA','VOMITOS',
'DOLOR Y/O MOLESTIAS CABEZA Y CUELLO','PRESION ARTERIAL ELEVADA',
'ACCIDENTE/TRAUMATISMO VIA PUBLICA','FIEBRE','CONVULSIONES',
'CONSTATACION DE LESIONES',
'ALCOHOLEMIA + CONST.LESIONES','DIARREA','AGRESION Y/O VIOLENCIA',
'MORDEDURA ANIMAL','MORDEDURA ARAÃA','QUEMADURAS','PROCEDIMIENTO DE HEMODINAMIA',
'ALCOHOLEMIA','HERIDAS Y/O HEMORRAGIA' ])

X[:,1] = le_motivo_consulta.transform(X[:,1]) 

In [9]:
X[0:]

array([[79, 18, 'POR SUS PROPIOS MEDIOS', ..., 11, 93, 268],
       [36, 18, 'SAMU MOVIL AVANZADO', ..., 3, 14, 209],
       [37, 18, 'POR SUS PROPIOS MEDIOS', ..., 2, 11, 10],
       ...,
       [55, 18, 'AMBULANCIA APS', ..., 5, 68, 2],
       [76, 18, 'SAMU MOVIL AVANZADO', ..., 3, 6, 713],
       [64, 18, 'POR SUS PROPIOS MEDIOS', ..., 13, 45, 11]], dtype=object)

Preprocesando medio de llegada

In [10]:
le_medio = preprocessing.LabelEncoder()
le_medio.fit([ 'POR SUS PROPIOS MEDIOS','SAMU MOVIL AVANZADO',
'AMBULANCIA APS','AMBULANCIA HOSPITAL','SAMU MOVIL BASICO',
'VEHICULO POLICIAL CARABINEROS','OTRO MEDIO TERRESTRE PAGADO',
'GENDARMERIA','AMBULANCIA CLINICAS PRIVADAS'])
X[:,2] = le_medio.transform(X[:,2])

Preprocesamiento Sexo

In [11]:
le_sexo = preprocessing.LabelEncoder()
le_sexo.fit([ 'FEMENINO','MASCULINO'])
X[:,3] = le_sexo.transform(X[:,3])

Preprocesamiento DM (Diabetes Mellitus)

In [12]:
le_DM = preprocessing.LabelEncoder()
le_DM.fit([ 'S', 'N','D'])
X[:,5] = le_DM.transform(X[:,5])

Preprocesamiento LCFA (Limintación crónica del flujo aéreo)

In [13]:
le_LCFA = preprocessing.LabelEncoder()
le_LCFA.fit([ 'S', 'N','D'])
X[:,8] = le_LCFA.transform(X[:,8]) 

Preprocesamiento HTA (Hipertención Arterial)

In [14]:
le_HTA = preprocessing.LabelEncoder()
le_HTA.fit([ 'S', 'N','D'])
X[:,10] = le_HTA.transform(X[:,10]) 

Preprocesamiento CAT ( Categoría)

In [15]:
le_CAT = preprocessing.LabelEncoder()
le_CAT.fit([ 'C1', 'C2','C3','C4','C5'])
X[:,11] = le_CAT.transform(X[:,11]) 

Asignación de la variable dependiente a predecir (categoría), al vector y

In [16]:
y = df_hosp["DESTINO"]

 <h1 id="Normalización de datos">Normalización de datos</h1>
<p>
Para aplicar el paquete de arboles de desición, los datos deben estar en una escala similar, es por ello que optamos por normalizarlos para que los valores estén en un rango entre -2 y 2. Esta medida no altera los resultados aunque si faborece a la eficiencia de los algoritmos
</p>

In [17]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[ 0.99625445,  0.42786127,  0.36170022, -0.85208079,  0.2006122 ,
         0.57425949, -0.64295632, -0.62778175,  3.3925378 , -1.01463125,
         0.65766514,  0.49536096,  0.10775771,  0.10775771, -0.39384883,
         0.84989103, -0.32068735, -0.46184096,  0.6603932 , -1.69078044,
         0.64321137, -1.02713707,  0.10775771,  0.10775771,  0.79010273,
         0.26093814],
       [-1.6282834 ,  0.42786127,  0.87119634,  1.17359764,  0.2006122 ,
        -1.54342888,  0.44213479, -0.70181777, -0.20863781, -0.14306879,
        -1.39202352,  0.49536096, -0.0672743 , -0.0672743 , -0.39384883,
         0.16082017, -0.39319425,  0.04833935,  1.53679199, -0.6976216 ,
        -0.34677353,  0.13840688, -0.0672743 , -0.0672743 , -0.55515844,
        -0.03713735],
       [-1.56724764,  0.42786127,  0.36170022,  1.17359764,  0.2006122 ,
         0.57425949, -0.64295632, -0.41625025, -0.20863781, -0.43358961,
        -1.39202352, -1.70128431, -0.0891533 , -0.0891533 , -0.39384883,
       

 <h1 id="Análisis de componentes principales">Análisis de componentes principales</h1>
<p>
Se realizó el análisis de componentes principales con el fin de reducir la dimensionalidad de la base de datos utilizada para la predicción de categoría. El objetivo de reducir la dimensionalidad de la base de datos es agilizar los procesos de entrenamiento y predicción de la categoria de pacientes, además de identificar las variables que presentan una mayor utilidad para esta, descartando las que no aportan indormación a la predicción.
</p>

In [23]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(X)
pca.explained_variance_ratio_
X.shape
#pca.n_components_ 

17

 <h1 id="Configurando algoritmos">Configurando algoritmos</h1>
<p>
En esta sección se definen parámetros necesarios para la correcta aplicación de los algoritmos a implementar, además de seccionar el conjunto de datos en datos de prueba(30%)y de entrenamiento (70%). Los parámetros escogidos pueden ser modificados con el fin de obtener resultados diferentes
</p>

Carga de paquetes

In [18]:
from sklearn.model_selection import train_test_split

Segmentación del conjunto de datos

In [19]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

Árbol de desición

In [20]:
from sklearn.tree import DecisionTreeClassifier
DT= DecisionTreeClassifier(criterion="entropy", max_depth = 4)
DT.fit(X_trainset,y_trainset)
yhat_1 = DT.predict(X_testset)
yhat_prob_1=DT.predict_proba(X_testset)
DT_Acc=round(metrics.accuracy_score(y_testset, yhat_1),4)
DT_Jcc=round(jaccard_similarity_score(y_testset, yhat_1),4)
DT_lgl=round(log_loss(y_testset, yhat_prob_1),4)
DT_F1=f1_score(y_testset, yhat_1, average='weighted') 

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [21]:
DT_parameters = [{'criterion': ['entropy', 'gini'], 'max_depth': np.arange(1, 21)},{'min_samples_leaf': [1, 5, 10, 20, 50, 100]}]
DT_GS = GridSearchCV(DecisionTreeClassifier(random_state=42), DT_parameters, verbose=True, n_jobs=-1, cv=3)
DT_GS.fit(X_trainset,y_trainset)

Fitting 3 folds for each of 46 candidates, totalling 138 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 138 out of 138 | elapsed:    4.8s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'criterion': ['entropy', 'gini'],
                    

In [22]:
DT_GS.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=100, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [23]:
yhat_1_GS= DT_GS.predict(X_testset)
yhat_prob_1_GS=DT_GS.predict_proba(X_testset)
DT_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_1_GS),4)
DT_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_1_GS),4)
DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS),4)
DT_F1_GS=round(f1_score(y_testset, yhat_1_GS, average='weighted'),4) 

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [24]:
resultados_DT = {'índices de rendimiento':['Accuracy','Jaccard','LogLoss','F1-Score'],
             'Árboles de decisión':[DT_Acc,DT_Jcc,DT_lgl,DT_F1],
             'Grid Search':[DT_Acc_GS,DT_Jcc_GS,DT_lgl_GS,DT_F1_GS]}
Tabla_resultados_DT=pd.DataFrame(resultados_DT)
print(Tabla_resultados_DT)

  índices de rendimiento  Árboles de decisión  Grid Search
0               Accuracy             0.405500       0.4001
1                Jaccard             0.405500       0.4001
2                LogLoss             1.636400       1.4113
3               F1-Score             0.316703       0.3577


Bosque de desición

In [25]:
from sklearn.ensemble import RandomForestClassifier
RF= RandomForestClassifier(max_depth=2, random_state=0)
RF.fit(X_trainset,y_trainset)
yhat_2 = RF.predict(X_testset)
yhat_prob_2=RF.predict_proba(X_testset)
RF_Acc=round(metrics.accuracy_score(y_testset, yhat_2),4)
RF_Jcc=round(jaccard_similarity_score(y_testset, yhat_2),4)
RF_lgl=round(log_loss(y_testset, yhat_prob_2),4)
RF_F1=f1_score(y_testset, yhat_2, average='weighted') 

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [26]:
RF_parameters ={ 'n_estimators': [200, 500],'max_features': ['auto', 'sqrt', 'log2'],'max_depth' : [2,3,4,5,6,7,8],'criterion' :['gini', 'entropy']}
RF_GS = GridSearchCV(estimator=RF,param_grid=RF_parameters, verbose=True, n_jobs=-1,cv=3)
RF_GS.fit(X_trainset,y_trainset)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=2,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=0,
                                      

In [27]:
RF_GS.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [29]:
yhat_2_GS= RF_GS.predict(X_testset)
yhat_prob_2_GS=RF_GS.predict_proba(X_testset)
RF_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_2_GS),4)
RF_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_2_GS),4)
RF_lgl_GS=round(log_loss(y_testset, yhat_prob_2_GS),4)
RF_F1_GS=round(f1_score(y_testset, yhat_2_GS, average='weighted'),4) 

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [30]:
resultados_RF = {'índices de rendimiento':['Accuracy','Jaccard','LogLoss','F1-Score'],
             'Bosque de decisión':[RF_Acc,RF_Jcc,RF_lgl,RF_F1],
             'Grid Search':[RF_Acc_GS,RF_Jcc_GS,RF_lgl_GS,RF_F1_GS]}
Tabla_resultados_RF=pd.DataFrame(resultados_RF)
print(Tabla_resultados_RF)

  índices de rendimiento  Bosque de decisión  Grid Search
0               Accuracy             0.37600       0.4216
1                Jaccard             0.37600       0.4216
2                LogLoss             1.29570       1.2584
3               F1-Score             0.27888       0.3761


Regresión logística (multi-class)

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='newton-cg',penalty='l2',multi_class='ovr').fit(X_trainset,y_trainset)
yhat_3= LR.predict(X_testset)
yhat_prob_3=LR.predict_proba(X_testset)
LR_Acc=round(metrics.accuracy_score(y_testset, yhat_3),4)
LR_Jcc=round(jaccard_similarity_score(y_testset, yhat_3),4)
LR_lgl=round(log_loss(y_testset, yhat_prob_3),4)
LR_F1=f1_score(y_testset, yhat_3, average='weighted') 

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [33]:
LR_parameters ={'C':np.logspace(-3,3,7), 'penalty':['l1','l2']}
LR_GS = GridSearchCV(estimator=LR,param_grid=LR_parameters, verbose=True, n_jobs=-1,cv=3)
LR_GS.fit(X_trainset,y_trainset)

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\jmoya\Anaconda3\lib\si

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=0.01, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='ovr',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='newton-cg',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [34]:
LR_GS.best_estimator_

LogisticRegression(C=1000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [38]:
yhat_3_GS= LR_GS.predict(X_testset)
yhat_prob_3_GS=LR_GS.predict_proba(X_testset)
LR_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_3_GS),4)
LR_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_3_GS),4)
LR_lgl_GS=round(log_loss(y_testset, yhat_prob_3_GS),4)
LR_F1_GS=round(f1_score(y_testset, yhat_3_GS, average='weighted'),4) 

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [39]:
resultados_LR = {'índices de rendimiento':['Accuracy','Jaccard','LogLoss','F1-Score'],
             'Regresión Logística':[LR_Acc,LR_Jcc,LR_lgl,LR_F1],
             'Grid Search':[LR_Acc_GS,LR_Jcc_GS,LR_lgl_GS,LR_F1_GS]}
Tabla_resultados_LR=pd.DataFrame(resultados_LR)
print(Tabla_resultados_LR)

  índices de rendimiento  Regresión Logística  Grid Search
0               Accuracy             0.402100       0.4028
1                Jaccard             0.402100       0.4028
2                LogLoss             1.281000       1.2826
3               F1-Score             0.353874       0.3626


Redes Neuronales

In [35]:
from sklearn.neural_network import MLPClassifier
NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1).fit(X_trainset,y_trainset)
yhat_4 = NN.predict(X_testset)
yhat_prob_4=NN.predict_proba(X_testset)
NN_Acc=round(metrics.accuracy_score(y_testset, yhat_4),4)
NN_Jcc=round(jaccard_similarity_score(y_testset, yhat_3),4)
NN_lgl=round(log_loss(y_testset, yhat_prob_3),4)
NN_F1=f1_score(y_testset, yhat_4, average='weighted') 

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [36]:
NN_parameters ={'solver': ['lbfgs'], 'max_iter': [500,1000,1500], 'alpha': 10.0 ** -np.arange(1, 7), 'hidden_layer_sizes':np.arange(5, 12), 'random_state':[0,1,2,3,4,5,6,7,8,9]}
NN_GS = GridSearchCV(estimator=NN,param_grid=NN_parameters, cv= 3, verbose=True, n_jobs=-1)
NN_GS.fit(X_trainset,y_trainset)

Fitting 3 folds for each of 1260 candidates, totalling 3780 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 17.5min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 24.7min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 33.3min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 43.5min
[Parallel(n_jobs=-1)]: Done 3780 out of 3780 | elapsed: 51.9min finished
C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_ite

GridSearchCV(cv=3, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=1e-05,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(5, 2),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=1...
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': array([1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06]),
               

In [40]:
NN_GS.best_estimator_

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=6, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=8, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [41]:
yhat_4_GS= NN_GS.predict(X_testset)
yhat_prob_4_GS=NN_GS.predict_proba(X_testset)
NN_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_4_GS),4)
NN_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_4_GS),4)
NN_lgl_GS=round(log_loss(y_testset, yhat_prob_4_GS),4)
NN_F1_GS=round(f1_score(y_testset, yhat_4_GS, average='weighted'),4) 

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [42]:
resultados_NN = {'índices de rendimiento':['Accuracy','Jaccard','LogLoss','F1-Score'],
             'Red Neuronal':[NN_Acc,NN_Jcc,NN_lgl,NN_F1],
             'Grid Search':[NN_Acc_GS,NN_Jcc_GS,NN_lgl_GS,NN_F1_GS]}
Tabla_resultados_NN=pd.DataFrame(resultados_NN)
print(Tabla_resultados_NN)

  índices de rendimiento  Red Neuronal  Grid Search
0               Accuracy      0.403500       0.3988
1                Jaccard      0.402100       0.3988
2                LogLoss      1.281000       1.3264
3               F1-Score      0.310289       0.3472


Support Vector Machine

In [43]:
from sklearn import svm
SVM = svm.SVC(kernel='rbf',decision_function_shape='ovo', probability=True)
SVM.fit(X_trainset, y_trainset) 
yhat_5 = SVM.predict(X_testset)
yhat_prob_5=SVM.predict_proba(X_testset)
SVM_Acc=round(metrics.accuracy_score(y_testset, yhat_5),4)
SVM_Jcc=round(jaccard_similarity_score(y_testset, yhat_5),4)
SVM_lgl=round(log_loss(y_testset, yhat_prob_5),4)
SVM_F1=f1_score(y_testset, yhat_5, average='weighted') 

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [45]:
SVM_parameters ={'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
SVM_GS = GridSearchCV(estimator=SVM,param_grid=SVM_parameters, cv= 3, verbose=True, n_jobs=-1)
SVM_GS.fit(X_trainset,y_trainset)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   54.1s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:  3.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovo', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [46]:
SVM_GS.best_estimator_

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [47]:
yhat_5_GS= SVM_GS.predict(X_testset)
yhat_prob_5_GS=SVM_GS.predict_proba(X_testset)
SVM_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_5_GS),4)
SVM_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_5_GS),4)
SVM_lgl_GS=round(log_loss(y_testset, yhat_prob_5_GS),4)
SVM_F1_GS=round(f1_score(y_testset, yhat_5_GS, average='weighted'),4) 

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [48]:
resultados_SVM = {'índices de rendimiento':['Accuracy','Jaccard','LogLoss','F1-Score'],
             'Red Neuronal':[SVM_Acc,SVM_Jcc,SVM_lgl,SVM_F1],
             'Grid Search':[SVM_Acc_GS,SVM_Jcc_GS,SVM_lgl_GS,SVM_F1_GS]}
Tabla_resultados_SVM=pd.DataFrame(resultados_SVM)
print(Tabla_resultados_SVM)

  índices de rendimiento  Red Neuronal  Grid Search
0               Accuracy      0.404200       0.3981
1                Jaccard      0.404200       0.3981
2                LogLoss      1.286500       1.2847
3               F1-Score      0.373516       0.3304


In [49]:
resultados = {'Algoritmos de clasificación':['Árboles de decisión','Bosques de decisión','Regresión Logística','Red Neuronal','Support Vector Machine'],
             'Accuracy':[DT_Acc_GS,RF_Acc_GS,LR_Acc_GS,NN_Acc_GS,SVM_Acc_GS],
             'Jaccard':[DT_Jcc_GS,RF_Jcc_GS,LR_Jcc_GS,NN_Jcc_GS,SVM_Jcc_GS],
             'LogLoss':[DT_lgl_GS,RF_lgl_GS,LR_lgl_GS,NN_lgl_GS,SVM_lgl_GS],
             'F1-Score':[DT_F1_GS,RF_F1_GS,LR_F1_GS,NN_F1_GS,SVM_F1_GS]}
Tabla_resultados=pd.DataFrame(resultados)
print(Tabla_resultados)

  Algoritmos de clasificación  Accuracy  Jaccard  LogLoss  F1-Score
0         Árboles de decisión    0.4001   0.4001   1.4113    0.3577
1         Bosques de decisión    0.4216   0.4216   1.2584    0.3761
2         Regresión Logística    0.4028   0.4028   1.2826    0.3626
3                Red Neuronal    0.3988   0.3988   1.3264    0.3472
4      Support Vector Machine    0.3981   0.3981   1.2847    0.3304


 <h1 id="Matrices de confusión">Matrices de confusión</h1>
<p>
En esta sección se construyen matrices de confusión para evaluar la eficiaca de cada algoritmo
</p>

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de confusión',
                          cmap=plt.cm.Blues):
    """
    Esta función muestra y dibuja la matriz de confusión.
    La normalización se puede aplicar estableciendo el valor `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusión normalizada")
    else:
        print('Matriz de confusión sin normalización')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Etiqueta Real')
    plt.xlabel('Etiqueta Predicha')


Árbol de decisión

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_1, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión árbol de decisión')

Bosque de decisión

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_2, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión bosque de decisión')

Regresión Logística

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_3, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión regresión logística')

Red neuronal

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_4, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión red neuronal')

Support Vector Machine

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_5, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión SVM')